## Hyperparam Tuning

Now that we know which models are performing better, it's time to perform cross validation and tune hyperparameters.
- Do a google search for hyperparameter ranges for each type of model.

GridSearch/RandomSearch are a great methods for checking off both of these tasks.

There is a fairly significant issue with this approach for this particular problem (described below). But in the interest of creating a basic functional pipeline, you can just use the default Sklearn methods for now.

## Preventing Data Leakage in Tuning - STRETCH

> **This step doesn't need to be part of your Minimum Viable Product (MVP), but its highly recommended you complete it, if you have time!**

BUT we have a problem - if we calculated a numerical value to encode city (such as the mean of sale prices in that city) on the training data, we can't cross validate 
- The rows in each validation fold were part of the original calculation of the mean for that city - that means we're leaking information!
- While sklearn's built in functions are extremely useful, sometimes it is necessary to do things ourselves

You need to create two functions to replicate what Gridsearch does under the hood. This is a challenging, real world data problem! To help you out, we've created some psuedocode and docstrings to get you started. 

**`custom_cross_validation()`**
- Should take the training data, and divide it into multiple train/validation splits. 
- Look into `sklearn.model_selection.KFold` to accomplish this - the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) shows how to split a dataframe and loop through the indexes of your split data. 
- Within your function, you should compute the city means on the training folds just like you did in Notebook 1 - you may have to re-join the city column to do this - and then join these values to the validation fold

This psuedocode may help you fill in the function:

```python
kfold = KFold() # fit sklearn k folds on X_train
train_folds = []
val_folds = []
for training_index, val_index in kfold.split(X_train):
    train_fold, val_fold = #.iloc loop variables on X_train

    # recompute training city means like you did in notebook 1 
    # merge to validation fold
        
    train_folds.append(train_fold)
    val_folds.append(val_fold)

    return train_folds, val_folds
```


**`hyperparameter_search()`**
- Should take the validation and training splits from your previous function, along with your dictionary of hyperparameter values
- For each set of hyperparameter values, fit your chosen model on each set of training folds, and take the average of your chosen scoring metric. [itertools.product()](https://docs.python.org/3/library/itertools.html) will be helpful for looping through all combinations of hyperparameter values
- Your function should output the hyperparameter values corresponding the highest average score across all folds. Alternatively, it could also output a model object fit on the full training dataset with these parameters.


This psuedocode may help you fill in the function:

```python
hyperparams = # Generate hyperparam options with itertools
hyperparam-scores = []
for hyperparam-combo in hyperparams:

    scores = []

    for folds in allmyfolds:
        # score fold the fold with the model/ hyperparams
        scores.append(score-fold)
        
    score = scores.mean()
    hyperparam-scores.append(score)
# After loop, find max of hyperparam-scores. Best params are at same index in `hyperparams` loop iteratble
```

Docstrings have been provided below to get you started. Once you're done developing your functions, you should move them to `functions_variables.py` to keep your notebook clean 

Bear in mind that these instructions are just one way to tackle this problem - the inputs and output formats don't need to be exactly as specified here.

In [61]:
# importing necessary packages
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, make_scorer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold
import itertools
import pickle
import os

In [63]:
# loading dataset
train_df = pd.read_csv('../preprocessed/train_df.csv')
test_df = pd.read_csv('../preprocessed/test_df.csv')

# X,y split
X_train = train_df.drop(columns=['description.sold_price'])
y_train = train_df['description.sold_price']
X_test = test_df.drop(columns=['description.sold_price'])
y_test = test_df['description.sold_price']

# removing outliers
index_outliers = y_test[y_test > 2000000].index
print(len(index_outliers))
X_test = X_test.drop(index_outliers)
y_test = y_test.drop(index_outliers)

y_train.isnull().sum()

0


0

## Hyperparam Tuning

In [64]:
# identifying feature types
numerical_features = X_train.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns
boolean_features = X_train.select_dtypes(include=['bool']).columns

# defining preprcessing steps
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

boolean_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='if_binary'))
])

# combing preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features),
        ('bool', boolean_transformer, boolean_features)
    ]
)

In [65]:
# defining rmse scoring function
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)
rmse_scorer = make_scorer(rmse, greater_is_better=False)

# defining model pipelines 
models = {
    'ridge': Ridge(),
    'random_forest': RandomForestRegressor(),
    'svm': SVR(),
    'gradient_boosting': GradientBoostingRegressor(),
    'xgboost': xgb.XGBRegressor()
}

param_grids = {
    'ridge': {
        'model__alpha': [0.1, 1, 10, 100]
    },
    'random_forest': {
        'model__n_estimators': [100, 200],
        'model__max_depth': [10, 20, None],
        'model__min_samples_split': [2, 5, 10]
    },
    'svm': {
        'model__C': [0.1, 1, 10],
        'model__kernel': ['linear', 'rbf'],
        'model__gamma': ['scale', 'auto']
    },
    'gradient_boosting': {
        'model__n_estimators': [100, 200],
        'model__learning_rate': [0.01, 0.1, 0.2],
        'model__max_depth': [3, 5, 7]
    },
    'xgboost': {
        'model__n_estimators': [100, 200],
        'model__learning_rate': [0.01, 0.1, 0.2],
        'model__max_depth': [3, 5, 7],
        'model__subsample': [0.8, 1.0],
        'model__colsample_bytree': [0.8, 1.0]
    }
}

In [66]:
# looping over models and performing gridsearchcv
best_estimators = {}
for model_name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        #('poly_features', PolynomialFeatures(degree=3)),
        ('model', model)
    ])
    
    # setting up gridsearchcv with cross validation
    '''
    grid_search = GridSearchCV(estimator=pipeline,
                              param_grid=param_grids[model_name],
                              cv=5,
                              scoring={'r2': 'r2', 'rmse': rmse_scorer},
                              n_jobs=-1,
                              refit='r2')
                              
    grid_search.fit(X_train, y_train)
    best_estimators[model_name] = grid_search.best_estimator_
    '''
    # setting up randomsearchcv with cross validation
    randomized_search = RandomizedSearchCV(estimator=pipeline,
                                           param_distributions=param_grids[model_name],  
                                           n_iter=72,  
                                           cv=5,  
                                           scoring={'r2': 'r2'},  
                                           refit='r2',  
                                           n_jobs=-1,  
                                           random_state=42)

    randomized_search.fit(X_train, y_train)
    best_estimators[model_name] = randomized_search.best_estimator_
    
    # best parameters and score for each model
    print(f"Best parameters for {model_name}: {randomized_search.best_params_}")
    print(f"Best cross-validated R2 for {model_name}: {randomized_search.best_score_:.5f}")

C:\Users\Natan\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=72. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for ridge: {'model__alpha': 0.1}
Best cross-validated R2 for ridge: 0.88022


C:\Users\Natan\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 18 is smaller than n_iter=72. Running 18 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for random_forest: {'model__n_estimators': 200, 'model__min_samples_split': 2, 'model__max_depth': None}
Best cross-validated R2 for random_forest: 0.98920


C:\Users\Natan\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 12 is smaller than n_iter=72. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for svm: {'model__kernel': 'linear', 'model__gamma': 'scale', 'model__C': 10}
Best cross-validated R2 for svm: 0.41067


C:\Users\Natan\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 18 is smaller than n_iter=72. Running 18 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for gradient_boosting: {'model__n_estimators': 100, 'model__max_depth': 7, 'model__learning_rate': 0.2}
Best cross-validated R2 for gradient_boosting: 0.98984
Best parameters for xgboost: {'model__subsample': 0.8, 'model__n_estimators': 200, 'model__max_depth': 7, 'model__learning_rate': 0.2, 'model__colsample_bytree': 0.8}
Best cross-validated R2 for xgboost: 0.99010


In [67]:
# retraining the best model on entire training set and evaluating on test set
for model_name, best_model in best_estimators.items():
    best_model.fit(X_train, y_train) 

    # predict on the test set
    y_pred = best_model.predict(X_test)

    # evaluate model performance
    test_r2 = r2_score(y_test, y_pred)
    test_rmse = rmse(y_test, y_pred)

    # print evaluation results
    print(f"Model: {model_name}")
    print(f"  Test R^2 Score: {test_r2:.5f}")
    print(f"  Test RMSE: {test_rmse:.2f}\n")

Model: ridge
  Test R^2 Score: 0.91252
  Test RMSE: 80994.67

Model: random_forest
  Test R^2 Score: 0.98982
  Test RMSE: 27628.42

Model: svm
  Test R^2 Score: 0.46317
  Test RMSE: 200645.40

Model: gradient_boosting
  Test R^2 Score: 0.98994
  Test RMSE: 27462.38

Model: xgboost
  Test R^2 Score: 0.98789
  Test RMSE: 30135.75



We want to make sure that we save our models.  In the old days, one just simply pickled (serialized) the model.  Now, however, certain model types have their own save format.  If the model is from sklearn, it can be pickled, if it's xgboost, for example, the newest format to save it in is JSON, but it can also be pickled.  It's a good idea to stay with the most current methods. 
- you may want to create a new `models/` subdirectory in your repo to stay organized

In [54]:
# saving best model
with open('../models/best_model.pk1', 'wb') as file:
    pickle.dump(grid_search.best_estimator_, file)

## Building a Pipeline (Stretch)

> **This step doesn't need to be part of your Minimum Viable Product (MVP), but its highly recommended you complete it if you have time!**

Once you've identified which model works the best, implement a prediction pipeline to make sure that you haven't leaked any data, and that the model could be easily deployed if desired.
- Your pipeline should load the data, process it, load your saved tuned model, and output a set of predictions
- Assume that the new data is in the same JSON format as your original data - you can use your original data to check that the pipeline works correctly
- Beware that a pipeline can only handle functions with fit and transform methods.
- Classes can be used to get around this, but now sklearn has a wrapper for user defined functions.
- You can develop your functions or classes in the notebook here, but once they are working, you should import them from `functions_variables.py` 

In [ ]:
# Build pipeline here

Pipelines come from sklearn.  When a pipeline is pickled, all of the information in the pipeline is stored with it.  For example, if we were deploying a model, and we had fit a scaler on the training data, we would want the same, already fitted scaling object to transform the new data with.  This is all stored when the pipeline is pickled.
- save your final pipeline in your `models/` folder

In [ ]:
# save your pipeline here